In [1]:
%load_ext autoreload
%autoreload 2

import torch
from torch.nn.functional import unfold, fold
from others.implementations import Convolution
from torch.nn.functional import conv2d

# Convolution

In [95]:
padding = 0
stride = 3
custom_conv = Convolution(3,1,kernel_size=2, padding=padding, stride = stride)

inp = torch.ones((5,3,5,5))
inp.requires_grad = True
out = custom_conv.forward(inp)
target = torch.zeros(out.size())

kernel = torch.ones((1,3,2,2))
kernel.requires_grad = True
kernel.retain_grad()
inp_padded = custom_conv.add_padding(inp, padding)
inp_padded = inp_padded.detach()
inp_padded.requires_grad = True
target_out = conv2d(inp_padded, kernel, stride=stride)
target_out.retain_grad()
loss = 0.5*((target-target_out)**2).sum()
loss.backward()
dl_dout = target_out.grad.detach()
dl_dw = kernel.grad.detach()
dl_dx = inp_padded.grad.detach()

c:\Users\stefa\deep-learning-project\Miniproject_2\others\implementations.py:39: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return ((prev_dim - kernel_dim)//stride)+1


In [96]:
#check out
(target_out - out).abs().sum()

tensor(0., grad_fn=<SumBackward0>)

In [99]:
#check dl_dx
(custom_conv.backward(dl_dout) - dl_dx).abs().sum()

torch.Size([5, 1, 2, 2])
torch.Size([5, 1, 6, 6])
torch.Size([3, 1, 2, 2])
torch.Size([5, 3, 5, 5])


tensor(0.)

In [100]:
#check dl_dw
(custom_conv.dl_dw - dl_dw).abs().sum()

tensor(0., grad_fn=<SumBackward0>)